<a href="https://colab.research.google.com/github/hesham81/AI/blob/developers/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





# PreProcessing Stage


Working With Testing Data

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download("punkt")
stemmer = PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
 return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  # pass


a = "How Long Does  shipping take ? "
print(a)
a = tokenize(a)
print(a)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


How Long Does  shipping take ? 


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [10]:
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
# Download the 'punkt_tab' resource if not already present
nltk.download('punkt_tab')
nltk.download("punkt")

stemmer = PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stem(word):
 return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
  tokenized_sentence = [stem(w) for w in tokenized_sentence]
  bag = np.zeros(len(all_words), dtype=np.float32)
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
  return bag



sentence = ["hello", "how", "are", "you"]
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
bag = bag_of_words(sentence, words)
print(bag)

[0. 1. 0. 1. 0. 0. 0.]


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Working With The Data Of Json
and also working with train
**Contain Creating The DataSet**

In [7]:
import json
from nltk_utils import *
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

with open('intents_training.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    # The problem was here. You need to iterate through intent['patterns'] not intents['patterns']
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['? ', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)
print(all_words)

x_train = []
y_train = []
for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  x_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)
x_train = np.array(x_train)
y_train = np.array(y_train)


class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(x_train)
    self.x_data = x_train
    self.y_data = y_train


  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples


batch_size = 8

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)





80 patterns
11 tags: ['accommodation', 'currency_exchange', 'dining', 'egypt_travel', 'emergency_services', 'goodbye', 'greeting', 'transportation', 'travel_booking', 'travel_tips', 'world_travel']
154 unique stemmed words: ['100', '?', 'a', 'about', 'abroad', 'accommod', 'advic', 'afford', 'afternoon', 'airbnb', 'alexandria', 'ambul', 'america', 'an', 'anyon', 'are', 'around', 'asia', 'aswan', 'attract', 'australia', 'avoid', 'bangkok', 'beach', 'best', 'book', 'budget-friendli', 'buse', 'bye', 'cafe', 'cairo', 'call', 'can', 'car', 'care', 'catch', 'cheap', 'cheapest', 'checklist', 'contact', 'convert', 'countri', 'cruis', 'currenc', 'deal', 'destin', 'dine', 'dish', 'do', 'dollar', 'dubai', 'easi', 'eat', 'egp', 'egypt', 'emerg', 'eur', 'euro', 'europ', 'even', 'exchang', 'family-friendli', 'find', 'fine', 'first-tim', 'flight', 'food', 'for', 'gbp', 'get', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'histor', 'hostel', 'hotel', 'how', 'i', 'in', 'inform', 'is', 'it', 'later', '

**Generating The Model**







In [11]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()
  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)


Using The Model

In [17]:
import json
from nltk_utils import *
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import NeuralNet

with open('intents_training.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    # The problem was here. You need to iterate through intent['patterns'] not intents['patterns']
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['? ', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)
print(all_words)

x_train = []
y_train = []
for (pattern_sentence, tag) in xy:
  bag = bag_of_words(pattern_sentence, all_words)
  x_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)
x_train = np.array(x_train)
y_train = np.array(y_train)


class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(x_train)
    self.x_data = x_train
    self.y_data = y_train


  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples


batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(x_train)
learning_rate = 0.001
num_epochs = 1000



dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(dtype=torch.long).to(device)

    outputs = model(words)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) % 100 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')






80 patterns
11 tags: ['accommodation', 'currency_exchange', 'dining', 'egypt_travel', 'emergency_services', 'goodbye', 'greeting', 'transportation', 'travel_booking', 'travel_tips', 'world_travel']
154 unique stemmed words: ['100', '?', 'a', 'about', 'abroad', 'accommod', 'advic', 'afford', 'afternoon', 'airbnb', 'alexandria', 'ambul', 'america', 'an', 'anyon', 'are', 'around', 'asia', 'aswan', 'attract', 'australia', 'avoid', 'bangkok', 'beach', 'best', 'book', 'budget-friendli', 'buse', 'bye', 'cafe', 'cairo', 'call', 'can', 'car', 'care', 'catch', 'cheap', 'cheapest', 'checklist', 'contact', 'convert', 'countri', 'cruis', 'currenc', 'deal', 'destin', 'dine', 'dish', 'do', 'dollar', 'dubai', 'easi', 'eat', 'egp', 'egypt', 'emerg', 'eur', 'euro', 'europ', 'even', 'exchang', 'family-friendli', 'find', 'fine', 'first-tim', 'flight', 'food', 'for', 'gbp', 'get', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'histor', 'hostel', 'hotel', 'how', 'i', 'in', 'inform', 'is', 'it', 'later', '

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/collate.py", line 211, in collate
    return [
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/collate.py", line 212, in <listcomp>
    collate(samples, collate_fn_map=collate_fn_map)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/collate.py", line 240, in collate
    raise TypeError

In [26]:
import json
from nltk_utils import *
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from model import NeuralNet

with open('intents_training.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    # The problem was here. You need to iterate through intent['patterns'] not intents['patterns']
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        # Skip empty patterns
        if not w:
            continue
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['? ', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)
print(all_words)

x_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

# Convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)


class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples


batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])  # Use the size of the bag-of-words vector
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)  # Labels are already long tensors

        outputs = model(words)
        # no_grad() is not needed here as we are in training mode
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')

80 patterns
11 tags: ['accommodation', 'currency_exchange', 'dining', 'egypt_travel', 'emergency_services', 'goodbye', 'greeting', 'transportation', 'travel_booking', 'travel_tips', 'world_travel']
154 unique stemmed words: ['100', '?', 'a', 'about', 'abroad', 'accommod', 'advic', 'afford', 'afternoon', 'airbnb', 'alexandria', 'ambul', 'america', 'an', 'anyon', 'are', 'around', 'asia', 'aswan', 'attract', 'australia', 'avoid', 'bangkok', 'beach', 'best', 'book', 'budget-friendli', 'buse', 'bye', 'cafe', 'cairo', 'call', 'can', 'car', 'care', 'catch', 'cheap', 'cheapest', 'checklist', 'contact', 'convert', 'countri', 'cruis', 'currenc', 'deal', 'destin', 'dine', 'dish', 'do', 'dollar', 'dubai', 'easi', 'eat', 'egp', 'egypt', 'emerg', 'eur', 'euro', 'europ', 'even', 'exchang', 'family-friendli', 'find', 'fine', 'first-tim', 'flight', 'food', 'for', 'gbp', 'get', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'histor', 'hostel', 'hotel', 'how', 'i', 'in', 'inform', 'is', 'it', 'later', '

TypeError: must be real number, not NoneType